In [1]:
import pandas as pd
import numpy as np
import json
import datetime 

In [116]:
sourceCheckins = pd.read_csv('data/July/server/checkins.tsv', sep='\t')
allCheckins = pd.read_csv('data/July/server/currentExtendedCheckinsHistory.tsv', sep='\t')
checkpoints = pd.read_csv('data/July/server/checkPoints.tsv', sep='\t')

In [8]:
len(np.unique(allCheckins.userId.tolist()))

3445

In [12]:
len(allCheckins)

371399

In [119]:
allCheckins = allCheckins.drop_duplicates(subset = ['checkinId'])

In [9]:
allCheckins = pd.concat([checkins1, checkins])

In [3]:
allCheckins = pd.read_csv('data/July/server/currentExtendedCheckinsHistory.tsv', sep='\t')

In [399]:
allCheckins.to_csv('data/July/server/currentExtendedCheckinsHistory.tsv', sep='\t', encoding='utf-8')

In [11]:
allCheckins.drop('Unnamed: 0', axis=1, inplace=True)

In [14]:
sortedVenues = []
sortedVenuesIds = []
for venueId, group in allCheckins.groupby('venueId').agg('size').order(ascending = False).iteritems():
    sortedVenues.append({'venueId':venueId,
                        'group':group})
    sortedVenuesIds.append(venueId)

In [115]:
#'World Class Lite in top ten!, 520f117111d2a782f1211c3b'

In [13]:
groupedUsers = allCheckins.groupby('userId')
userVenues = []
for index, group in enumerate(groupedUsers):
        userVenues.append(group[1]['venueId'].tolist())

In [15]:
len(sortedVenues)

83292

In [16]:
top50VenuesCheckins = allCheckins[allCheckins.venueId.isin(sortedVenuesIds[:50])]

In [17]:
for venueId in sortedVenuesIds[:50]:
    print venueNameFromId(venueId)

NameError: name 'venueNameFromId' is not defined

In [ ]:
from pymining import itemmining
relim_input = itemmining.get_relim_input(userVenues)
report = itemmining.relim(relim_input, min_support=2)
report

In [51]:
def load_dataset():
    "Load the sample dataset."
    return userVenues
 
 
def createC1(dataset):
    "Create a list of candidate item sets of size one."
    c1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    #frozenset because it will be a ket of a dictionary.
    return map(frozenset, c1)
 
 
def scanD(dataset, candidates, min_support):
    "Returns all candidates that meets a minimum support level"
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1
 
    num_items = float(len(dataset))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support
    return retlist, support_data
 
 
def aprioriGen(freq_sets, k):
    "Generate the joint transactions from candidate sets"
    retList = []
    lenLk = len(freq_sets)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(freq_sets[i])[:k - 2]
            L2 = list(freq_sets[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(freq_sets[i] | freq_sets[j])
    return retList
 
 
def apriori(dataset, minsupport=0.5):
    "Generate a list of candidate item sets"
    C1 = createC1(dataset)
    D = map(set, dataset)
    L1, support_data = scanD(D, C1, minsupport)
    L = [L1]
    k = 2
    while (len(L[k - 2]) > 0):
        Ck = aprioriGen(L[k - 2], k)
        Lk, supK = scanD(D, Ck, minsupport)
        support_data.update(supK)
        L.append(Lk)
        k += 1
 
    return L, support_data

In [ ]:
C1 = createC1(userVenues)

In [252]:
D = map(set, userVenues)

In [253]:
L1, support_data = scanD(D, C1, 0.01)

In [53]:
L, supportData = apriori(userVenues, minsupport=0.01)

In [54]:
frequentVenues = []
for itemset, support in zip(L, supportData):
    if len(itemset)>5:
        #print support
        frequentVenueSet = []
        for item in itemset:
            frequentVenueSet.append(next(enumerate(item))[1])
        frequentVenues.append(frequentVenueSet)

In [40]:
len(userVenues)*0.0015455950541

1.0000000000027

In [55]:
for venueId in frequentVenues[0][:20]:
    print venueNameFromId(venueId)

Афимолл Сити / Afimall City
ТРЦ РИО «Ленинский»
Гин-но Таки
McDonald's
Cuba Libre
Hard Rock Cafe Moscow
Conversation Cafe
Bar BQ Cafe
Каро Vegas 22
Счастье
Площадь Восстания
Starbucks
Аэроэкспресс - Терминал на Павелецком вокзале
Osteria della Piazza Bianca
ТРK «Вегас» (Крокус Сити)
Biblioteka
ТК «Горбушка»
Солнечногорск
Багратион
Государственная Третьяковская галерея на Крымском Валу


In [56]:
def generateRules(L, support_data, min_confidence=0.7):
    """Create the association rules
    L: list of frequent item sets
    support_data: support data for those itemsets
    min_confidence: minimum confidence threshold
    """
    rules = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            #print "freqSet", freqSet, 'H1', H1
            if (i > 1):
                rules_from_conseq(freqSet, H1, support_data, rules, min_confidence)
            else:
                calc_confidence(freqSet, H1, support_data, rules, min_confidence)
    return rules
 
 
def calc_confidence(freqSet, H, support_data, rules, min_confidence=0.7):
    "Evaluate the rule generated"
    pruned_H = []
    for conseq in H:
        conf = support_data[freqSet] / support_data[freqSet - conseq]
        if conf >= min_confidence:
            print freqSet - conseq, '--->', conseq, 'conf:', conf
            rules.append((freqSet - conseq, conseq, conf))
            pruned_H.append(conseq)
    return pruned_H
 
 
def rules_from_conseq(freqSet, H, support_data, rules, min_confidence=0.7):
    "Generate a set of candidate rules"
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calc_confidence(freqSet, Hmp1,  support_data, rules, min_confidence)
        if len(Hmp1) > 1:
            rules_from_conseq(freqSet, Hmp1, support_data, rules, min_confidence)

In [ ]:
rules = generateRules(L, supportData, min_confidence=0.1)

# Helper Functions

In [18]:
venuesDict = {}
def venueNameFromId(venueId):
    if venueId not in venuesDict.keys():
        venuesDict[venueId] = allCheckins[allCheckins.venueId == venueId].iloc[0]['name']
    return venuesDict[venueId]

In [19]:
venuesCatDict = {}
def venueCategoryFromId(venueId):
    if venueId not in venuesCatDict:
        venuesCatDict[venueId] = allCheckins[allCheckins.venueId == venueId].iloc[0]['categoryName']
    return venuesCatDict[venueId]

In [20]:
venuesCatIdsDict = {}
def venueCategoryIdFromName(categoryName):
    if categoryName not in venuesCatIdsDict:
        venuesCatIdsDict[categoryName] = allCheckins[allCheckins.categoryName == categoryName].iloc[0]['categoryId']
    return venuesCatIdsDict[categoryName]

In [21]:
categoriesDict = {}
def categoryNameFromId(categoryId):
    if categoryId not in categoriesDict.keys():
        categoriesDict[categoryId] = allCheckins[allCheckins.categoryId == categoryId].iloc[0]['categoryName']
    return categoriesDict[categoryId]

In [22]:
def countUsersAtVenue(venuedId):
    return len(np.unique(allCheckins[allCheckins.venueId == venueId]['userId'].tolist()))

In [68]:
for rule in rules:
    simpleRule = 1
    if len(rule[0]) < 2:
        continue
        
    for venueId in enumerate(rule[0]):
        if venueId[1] not in sortedVenuesIds[:20]:
            simpleRule = 0
            
    if simpleRule:
        continue
    
    simpleRule = 1
    
    for venueId in enumerate(rule[1]):
        if venueId[1] not in sortedVenuesIds[:20]:
            simpleRule = 0
            
    if simpleRule:
        continue
    
    for venueId in enumerate(rule[0]):
        print venueNameFromId(venueId[1])
        
    print '--->'
    for venueId in enumerate(rule[1]):
        print venueNameFromId(venueId[1])
    print 'conf:', rule[2]
    print ''

Ленинградский вокзал / Leningradsky Railway Terminal
Дворцовая площадь / Palace Square
--->
Невский проспект / Nevsky Prospect
Московский вокзал / Moskovsky Railway Station
conf: 0.395833333333

Ленинградский вокзал / Leningradsky Railway Terminal
Невский проспект / Nevsky Prospect
--->
Дворцовая площадь / Palace Square
Московский вокзал / Moskovsky Railway Station
conf: 0.520547945205

Дворцовая площадь / Palace Square
Московский вокзал / Moskovsky Railway Station
--->
Ленинградский вокзал / Leningradsky Railway Terminal
Невский проспект / Nevsky Prospect
conf: 0.351851851852

Невский проспект / Nevsky Prospect
Московский вокзал / Moskovsky Railway Station
--->
Ленинградский вокзал / Leningradsky Railway Terminal
Дворцовая площадь / Palace Square
conf: 0.463414634146



In [ ]:
#Using code for rules from here: http://aimotion.blogspot.ru/2013/01/machine-learning-and-data-mining.html

In [268]:
multiUserSortedVenues = filter(lambda x: x['group'] > 1, sortedVenues)

In [23]:
userVenueGroups = []
for venueId, group in allCheckins.groupby('venueId'):
    if len(group.groupby('userId'))>10:
        userVenueGroups.append(venueId)

In [24]:
userVenueCategories = []
for categoryId, group in allCheckins.groupby('categoryId'):
    if len(group.groupby('userId'))>10:
        userVenueCategories.append(categoryId)

In [25]:
#Make lists of users check-in venues
multiUserCheckins = allCheckins[allCheckins.venueId.isin(userVenueGroups)]
groupedUsers = multiUserCheckins.groupby('userId')
userVenuesList = []
userIds = []
for index, group in enumerate(groupedUsers):
        userVenuesList.append(group[1]['venueId'].tolist())
        userIds.append(group[0])

In [26]:
#Make lists of users check-in catrgories
fineCheckins = allCheckins[allCheckins.categoryId.isin(userVenueCategories)]
groupedUsers = fineCheckins.groupby('userId')
userCategoryList = []
userIdsForCat = []
for index, group in enumerate(groupedUsers):
        userCategoryList.append(group[1]['categoryId'].tolist())
        userIdsForCat.append(group[0])

In [27]:
len(userVenueGroups)

2544

# K-means, spectral

In [28]:
#Let's build some TF-ID vectors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.cluster import MiniBatchKMeans

In [29]:
venuesByUser = []
for userList in userVenuesList:
    venuesByUser.append(' '.join(userList))
    
categoriesByUser = []
for userList in userCategoryList:
    try:
        categoriesByUser.append(' '.join(userList))
    except:
        print userList

tfidf_vectorizer = TfidfVectorizer(vocabulary = userVenueGroups)
tfidf_matrix_train = tfidf_vectorizer.fit_transform(venuesByUser)

In [30]:
from time import time
cocluster = SpectralCoclustering(n_clusters=20,
                                 svd_method='arpack', random_state=0)
kmeans = MiniBatchKMeans(n_clusters=20,
                         random_state=0)

print("Coclustering...")
start_time = time()
cocluster.fit(tfidf_matrix_train)
y_cocluster = cocluster.row_labels_
print("Done in {:.2f}s.".format(
    time() - start_time))

print("MiniBatchKMeans...")
start_time = time()
y_kmeans = kmeans.fit_predict(tfidf_matrix_train)
print("Done in {:.2f}s.".format(
    time() - start_time))

feature_names = tfidf_vectorizer.get_feature_names()

Coclustering...
Done in 0.57s.
MiniBatchKMeans...
Done in 0.11s.


In [41]:
clustersSpectral = [[] for i in range(20)]
for index, clusterId in enumerate(y_cocluster):
    clustersSpectral[clusterId].append(userIds[index])

In [42]:
np.std([len(cluster) for cluster in  clusters])

NameError: name 'clusters' is not defined

In [43]:
clusters = x = [[] for i in range(20)]
for index, clustedId in enumerate(y_kmeans):
    clusters[clustedId].append(userIds[index])

# LDA

In [31]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer 

cv = sklearn.feature_extraction.text.CountVectorizer()
mat = cv.fit_transform(venuesByUser).toarray()
print('Done Document matrix')

Done Document matrix


In [32]:
sum([len(venuesUser) for venuesUser in userVenuesList])

135164

In [33]:
cvCategory = sklearn.feature_extraction.text.CountVectorizer()
matCategory = cvCategory.fit_transform(categoriesByUser).toarray()
print('Done Document matrix')

Done Document matrix


In [34]:
len(userVenueCategories)

461

In [69]:
venuesVocab = cv.get_feature_names()
print len(venuesVocab)

2544


In [36]:
categoriesVocab = cvCategory.get_feature_names()
print len(categoriesVocab)

461


In [37]:
import lda
model20 = lda.LDA(n_topics=20, n_iter=2000, random_state=1,alpha = 0.001)
model20.fit(mat)  # model.fit_transform(X) is also available
topic_word = model20.topic_word_  # model.components_ also works
print('Done LDA fitting')

Done LDA fitting


In [65]:
import lda
model20Cat = lda.LDA(n_topics=20, n_iter=2000, random_state=1,alpha = 0.001)
model20Cat.fit(matCategory)  # model.fit_transform(X) is also available
topic_cats = model20Cat.topic_word_  # model.components_ also works
print('Done LDA fitting')

Done LDA fitting


In [38]:
n_top_venues = 10
for i, venues_dist in enumerate(topic_word):
    indicies = np.argsort(venues_dist)[::-1]
    topic_venues = np.array(venuesVocab)[indicies]
    topic_probs = venues_dist[indicies]
    #print np.sort(venues_dist)[:-n_top_venues:-1]
    print('Cluster {}:'.format(i+1))
    for venueId, topicProb in zip(topic_venues, topic_probs):
        if topicProb>=0.01:
            print("{}, {} users {:.2f}%").format(venueNameFromId(venueId),countUsersAtVenue(venueId),topicProb*100)
        else:
            break
    print ''

Cluster 1:
ВДНХ (Выставка достижений народного хозяйства), 994 users 4.58%
ЦПКиО им. Горького / Gorky Park, 1728 users 2.90%
ТРЦ «Золотой Вавилон», 208 users 2.87%
Ярославский вокзал, 193 users 2.22%
Красная площадь / Red Square, 755 users 2.15%

Cluster 2:
Физика, 44 users 6.81%
Афимолл Сити / Afimall City, 455 users 6.30%
ТРЦ «Европейский», 539 users 4.80%
ТРЦ «Филион», 110 users 3.60%
ТЦ «Кунцево Плаза», 66 users 2.81%
ПКиО «Фили», 144 users 2.59%
Парк Победы (Поклонная гора) / Victory Park, 366 users 2.42%
Арбат / Arbat Street, 557 users 2.21%
ALEX-Fitness, 24 users 1.99%
ЦПКиО им. Горького / Gorky Park, 1728 users 1.51%
ТЦ «ЕвроПарк» / EuroPark Mall, 59 users 1.39%
Башня «Федерация» / Federation Tower, 49 users 1.32%
Район «Крылатское», 23 users 1.01%

Cluster 3:
Москва / Moscow, 656 users 38.49%
Россия / Russia, 321 users 18.77%
Западный административный округ, 148 users 3.11%
Северный административный округ, 95 users 2.26%
ЦПКиО им. Горького / Gorky Park, 1728 users 1.98%
Красна

# Create LDA users clusters

In [39]:
def exportLDAClustersToCSV(venues_clusters, allCheckins):
    topVenuesLDA = []
    LDAclustersDF = pd.DataFrame()
    for clustedId, venues_dist in enumerate(venues_clusters):
        indicies = np.argsort(venues_dist)[::-1]
        
        topic_venues = np.array(venuesVocab)[indicies]
        topic_probs = venues_dist[indicies]

        topVenuesIds =  [venueId for venueId, topicProb in zip(topic_venues, topic_probs) if topicProb>=0.01]
        topVenuesLDA.extend(topVenuesIds)
        clusterCheckins = allCheckins[allCheckins.venueId.isin(topVenuesIds)]
        clusterCheckins["clusterId"] = clustedId+1
        LDAclustersDF = LDAclustersDF.append(clusterCheckins)
    print len(topVenuesLDA)
    print len(set(topVenuesLDA))
    return LDAclustersDF

In [40]:
LDAclustersDF = exportLDAClustersToCSV(topic_word, allCheckins)

271
206


/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [41]:
LDAclustersDF.to_csv('data/July/LDATopVenuesMoreThan10Whole.tsv', sep="\t", encoding="utf-8")

In [46]:
LDAclusteredUsersDF = pd.DataFrame()
for clusterId, clusterLDA in enumerate(clustersLDA):
    clusterCheckins = allCheckins[allCheckins.userId.isin(clusterLDA)]
    clusterCheckins["clusterId"] = clusterId + 1
    LDAclusteredUsersDF = LDAclusteredUsersDF.append(clusterCheckins)
    print len(clusterLDA)

250
149
114
218
235
38
41
78
620
333
101
87
138
263
61
270
264
51
92
42


/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [48]:
LDAclusteredUsersDF.to_csv('data/July/LDAClusteredUsersMoreThan10Whole.tsv', sep="\t", encoding="utf-8")

In [47]:
from datetime import datetime
def extractComponents(row):
    date = datetime.utcfromtimestamp(row.createdAt+180*60)
    hours.append(date.hour)
    weekdays.append(date.weekday())
    
hours = []
weekdays = []

LDAclusteredUsersDF.apply(extractComponents, axis=1)
LDAclusteredUsersDF['hour'] = hours
LDAclusteredUsersDF['weekday'] = weekdays

In [476]:
for index, group in enumerate(LDAclusteredUsersDF.groupby("clusterId")):
    print 'Checkins per user: ', len(group[1][group[1]["createdAtDate"][6:7] == '6'])/len(group[1].groupby("userId"))

 Checkins per user: 

IndexingError: Unalignable boolean Series key provided

In [45]:
for clusterLDA,clusterLDACat  in zip(clustersLDA,clustersLDACategory):
    print len(clusterLDA), len(clusterLDACat)

NameError: name 'clustersLDACategory' is not defined

In [50]:
date = datetime.now()
date.weekday()

1

In [109]:
LDACheckins = allCheckins[allCheckins.venueId.isin(venuesVocab)]
print len(LDACheckins)

134866


In [ ]:
LDAclusteredCategoriesDF = pd.DataFrame()
for clusterId, clusterLDA in enumerate(clustersLDACategory):
    clusterCheckins = LDACheckins[LDACheckins.userId.isin(clusterLDA)]
    clusterCheckins["clusterId"] = clusterId + 1
    LDAclusteredCategoriesDF = LDAclusteredCategoriesDF.append(clusterCheckins)
    print len(clusterCheckins)
LDAclusteredCategoriesDF.to_csv('data/July/LDAClusteredUsersByCategory.tsv', sep="\t", encoding="utf-8")

In [44]:
clustersLDA = [[] for i in range(20)]
for index, userTopics in enumerate(model20.ndz_):
    clustersLDA[np.argmax(userTopics)].append(userIds[index])

In [102]:
clustersLDACategory = [[] for i in range(20)]
for index, userTopics in enumerate(model20Cat.ndz_):
    clustersLDACategory[np.argmax(userTopics)].append(userIdsForCat[index])

In [113]:
#Match the clusters by min Jaccard Distance
import distance
LDAClustersMap = []
LDAClustersMapIndexes = []
# clustersLDACopy = clustersLDA
distancesMatrix = []
for clusterK in clusters:
    distances = [distance.jaccard(clusterLDA,clusterK) for clusterLDA in clustersLDA]
    distancesMatrix.append(distances)
    index = np.argmin(distances)
    LDAClustersMap.append(clustersLDA[index])
#     clustersLDACopy.remove(clustersLDA[index])

In [156]:
clustersSpectral1 = [[0] if len(cluster) == 0 else cluster for cluster in clusterSpectreal]

In [ ]:
distancesMatrixKSpect = []
for clusterK in clusters:
    distances = [distance.jaccard(clusterSpectreal,clusterK) for clusterSpectreal in clustersSpectral]
    distancesMatrixKSpect.append(distances)

In [114]:
#Average Jaccard Distance between clusters
print np.mean([distance.jaccard(clusterK, LDAClustersMap[index]) for index, clusterK in enumerate(clusters) if len(clusterK)])

0.891365675713


In [124]:
#Average Jaccard Distance between clusters
print np.mean(np.min(distancesMatrix, axis=1))

0.923955972999


In [89]:
n_top_venues = 10
for i, venues_dist in enumerate(topic_cats):
    indicies = np.argsort(venues_dist)[:-n_top_venues:-1]
    topic_venues = np.array(categoriesVocab)[indicies]
    topic_probs = venues_dist[indicies]
    print('Cluster {}:'.format(i+1))
    for categoryId, topicProb in zip(topic_venues, topic_probs):
        if topicProb>=0.01:
            print("{}, {:.2f}%").format(categoryNameFromId(categoryId),topicProb*100)
        else:
            break
    print ''

Cluster 1:
Residential Building (Apartment / Condo), 10.83%
Mall, 10.52%
Coffee Shop, 9.17%
Bank, 6.00%
Park, 5.05%
Fast Food Restaurant, 4.54%
Grocery Store, 2.16%
Big Box Store, 2.15%
Sushi Restaurant, 1.95%

Cluster 2:
Park, 5.66%
Coffee Shop, 5.09%
Airport, 4.82%
Mall, 4.81%
Hotel, 3.55%
Café, 3.08%
Train Station, 2.73%
City, 2.12%
Restaurant, 1.76%

Cluster 3:
Office, 19.11%
Gym / Fitness Center, 17.62%
Tech Startup, 6.05%
Airport, 3.92%
Cocktail Bar, 2.27%
Park, 1.97%
Medical Center, 1.97%
Mall, 1.88%
Hotel, 1.83%

Cluster 4:
Home (private), 43.82%
Office, 4.58%
Mall, 4.33%
Residential Building (Apartment / Condo), 3.73%
Gym Pool, 2.25%
Capitol Building, 1.96%
Financial or Legal Service, 1.37%
Airport, 1.33%
Clothing Store, 1.29%

Cluster 5:
Gym / Fitness Center, 66.53%
Mall, 6.70%
College Communications Building, 1.96%
Train Station, 1.67%
Park, 1.56%
Salon / Barbershop, 1.56%
Multiplex, 1.54%
Hockey Arena, 1.11%
Sushi Restaurant, 1.06%

Cluster 6:
Park, 16.31%
Mall, 5.01%
Stree

In [228]:
print len(set(multiUserCheckins[multiUserCheckins.userId == clusters[10][1]]['venueId'].tolist()).intersection(multiUserCheckins[multiUserCheckins.userId == clusters[10][2]]['venueId'].tolist()))
print len(set(multiUserCheckins[multiUserCheckins.userId == clusters[10][1]]['venueId'].tolist()).union(multiUserCheckins[multiUserCheckins.userId == clusters[10][2]]['venueId'].tolist()))

4
140


In [13]:
import schedule
import time

schedule.clear()
schedule.every(2).seconds.do(saveDate)

while True:
    schedule.run_pending()
    time.sleep(1)

2015-07-10 00:56:53.189411
2015-07-10 00:58:12.707836
2015-07-10 00:58:14.715938
2015-07-10 00:58:16.722812


KeyboardInterrupt: 

In [12]:
def saveDate():
    global currentDate
    print currentDate
    currentDate = datetime.now()

In [6]:
from datetime import datetime
currentDate = datetime.now()

In [181]:
type(allCheckins[:2]["createdAtDate"][0]
     from datetime import datetime
def extractComponents(row):
    date = datetime.utcfromtimestamp(row.createdAt+180*60)
    hours.append(date.hour)
    weekdays.append(date.weekday())

str

# Code for Visualization

In [281]:
#Make lists of users check-in venues
groupedUsersList = allCheckins.groupby('userId')
TMUserVenuesList = []
TMUsersCategoriesList = []
TMUserIds = []

for index, group in enumerate(groupedUsersList):
    sortedCheckins = group[1].sort("createdAt")
    venuePairs = []
    categoryPairs = []
    for venue1, venue2 in pairwise(sortedCheckins['venueId'].tolist()):
        venuePairs.append((venue1, venue2))
        categoryPairs.append((venueCategoryFromId(venue1),venueCategoryFromId(venue2)))
    timeDiffs = []
    for checkin1, checkin2 in pairwise(sortedCheckins['createdAt'].tolist()):
        timeDiffs.append((checkin2-checkin1)/60)
    TMUserVenuesList.append(zip(venuePairs,timeDiffs))
    TMUsersCategoriesList.append(zip(categoryPairs,timeDiffs))
    TMUserIds.append(group[0])

In [496]:
from sys import stdout
def progress(i, n):
    stdout.write("\r%f%%" % (i*100/float(n)))
    stdout.flush()
    if i == n-1:
        stdout.write("\r100%")
        print("\r\n")

In [518]:
def getSequencesFromCheckins(checkinsDF):
    checkinsGroupedByUser = checkinsDF.groupby('userId')
    venuePairsGroupedByUser = []
    categoryPairsGroupedByUser = []
    userIds = []
    count = len(checkinsGroupedByUser)

    for index, group in enumerate(checkinsGroupedByUser):
        progress(index, count)
        sortedCheckins = group[1].sort("createdAt")
        
        venuePairs = []
        categoryPairs = []
        
        for venue1, venue2 in pairwise(sortedCheckins['venueId'].tolist()):
            venuePairs.append((venue1, venue2))
            categoryPairs.append((venueCategoryFromId(venue1), venueCategoryFromId(venue2)))
            
        timeDiffs = []
        for checkin1, checkin2 in pairwise(sortedCheckins['createdAt'].tolist()):
            timeDiffs.append((checkin2-checkin1)/60)
            
        venuePairsGroupedByUser.append(zip(venuePairs,timeDiffs))
        categoryPairsGroupedByUser.append(zip(categoryPairs,timeDiffs))
        userIds.append(group[0])
    return (venuePairsGroupedByUser, categoryPairsGroupedByUser, userIds)

In [526]:
(venuePairsGroupedByUser5, categoryPairsGroupedByUser5, userIds5) = getSequencesFromCheckins(LDAclusteredUsersDF[LDAclusteredUsersDF.clusterId == 5])

100%



In [258]:
averageTimes = [np.mean([transitionTuple[1] for transitionTuple in userTransitions])/60 for userTransitions in TMUserVenuesList]

In [533]:
import collections
def getCategoryPredcessorsCounter(categoryName, categoryPairs):
    predcessorsByUser = [[transitionTuple[0][0] for transitionTuple in userTransitions if transitionTuple[0][1] == categoryName and transitionTuple[1] < 60*6] for userTransitions in categoryPairs]
    predcessorsList = [item for sublist in predcessorsByUser for item in sublist]
    return collections.Counter(predcessorsList)

def getCategorySuccessorsCounter(categoryName, categoryPairs):
    successorsByUser = [[transitionTuple[0][1] for transitionTuple in userTransitions if transitionTuple[0][0] == categoryName and transitionTuple[1] < 60*6] for userTransitions in categoryPairs]
    successorsList =  [item for sublist in successorsByUser for item in sublist]
    return collections.Counter(successorsList)

In [548]:
categoryCounter5 = getCategoryPredcessorsCounter('Coffee Shop', categoryPairsGroupedByUser5)

In [565]:
saveTopSuccessorsColorsJSON('Coffee Shop', categoryPairsGroupedByUser5, 'Production/vis/colors5.json', 10)

In [564]:
def getTopSuccessorsNames(categoryName, categoryPairs, nTop = 10):
    topNames = [commonTuple[0] for commonTuple in  getCategoryPredcessorsCounter(categoryName, categoryPairs).most_common(nTop)]
    return topNames
def saveTopSuccessorsColorsCSV(categoryName, categoryPairs, filename, nTop = 10):
    topNames = getTopSuccessorsNames(categoryName, categoryPairs, nTop)
    nameColorList = pd.DataFrame({'name':topNames,
    'color':[
    '#3182bd',
    '#e6550d',
    '#fdae6b',
    '#31a354',
    '#756bb1',
    '#636363',
    '#d6616b',
    '#7b4173',
    '#bd9e39',
    #'#b5cf6b',
    '#a1d99b']})
    nameColorList.to_csv(filename, encoding="utf-8", index=None)
    
def saveTopSuccessorsColorsJSON(categoryName, categoryPairs, filename, nTop = 10):
    topNames = getTopSuccessorsNames(categoryName, categoryPairs, nTop)
    jsonOfColors = [dict(zip(['name', 'color'],row)) for row in izip(topNames,[
    '#3182bd',
    '#e6550d',
    '#fdae6b',
    '#31a354',
    '#756bb1',
    '#636363',
    '#d6616b',
    '#7b4173',
    '#bd9e39',
    #'#b5cf6b',
    '#a1d99b'])]
    with open(filename, 'w') as outfile:
        json.dump(jsonOfColors, outfile)

In [546]:
def getMatrixForTopSuccessors(categoryName, categoryPairs, nTop = 10, withOthers = 1):
    #Get the counter and list of successors
    categoryCounter = getCategorySuccessorsCounter(categoryName, categoryPairs)
    categoryTopSuccessors = categoryCounter.most_common(nTop)
    topSuccessorsNames = [commonTuple[0] for commonTuple in categoryTopSuccessors]
    
    #Calculate total sums of inputs/outputs
    totalOut = sum(categoryCounter.values())
    totalTopCatsInterOut = 0
    totalIn = sum(getCategoryPredcessorsCounter(categoryName, categoryPairs).values())
    
    #+1 for Others section
    adjMatrix = [[] for i in range(nTop + withOthers)]
    
    #Fill the first element with the self value
    adjMatrix[0] = [0 for i in range(nTop + withOthers)]
    adjMatrix[0][0] = categoryCounter[categoryName]
    
    index = 1
    for commonTuple in  categoryTopSuccessors:
        successorCategory = commonTuple[0]
        if successorCategory != categoryName:
            #Count of transitions from source category
            adjMatrix[0][index] = commonTuple[1]
            
            successorCategoryCounter = getCategorySuccessorsCounter(successorCategory, categoryPairs)
            #Counts for all the important categories
            adjRow = [0 for i in range(nTop)]
            adjRow[0] = successorCategoryCounter[categoryName]
            adjRow[1:] = [successorCategoryCounter[catName] for catName in topSuccessorsNames if catName!=categoryName]
            #Others set to 0
            if withOthers:
                adjRow.append(0)
            adjMatrix[index] = adjRow
            totalTopCatsInterOut  = totalTopCatsInterOut + sum(adjRow)
            index = index + 1
    if withOthers:
        adjMatrix[0][nTop] = totalOut - sum(adjMatrix[0])
        adjMatrix[nTop] = [0 for i in range(nTop + withOthers)]
        adjMatrix[nTop][0] = totalIn - sum(row[0] for row in adjMatrix)
    
    #Normalize
    total = sum([sum(sublist) for sublist in adjMatrix])
    normMatrix = [[float(count)/total for count in adjRow] for adjRow in adjMatrix]
    return normMatrix

In [559]:
with open('Vis/coffeeMatrix5.json', 'w') as outfile:
    json.dump(getMatrixForTopSuccessors('Coffee Shop', categoryPairsGroupedByUser5, 10, 0), outfile)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
averageTimesFiltered = [averageTime for averageTime in averageTimes if averageTime < 24*7]
print int(float(len(averageTimesFiltered))/len(averageTimes)*100),"%"
plt.hist(averageTimesFiltered, bins = 48-0.5)

In [ ]:
averageTimesFiltered = [averageTime for averageTime in averageTimes if averageTime >= 24*7 and averageTime <= 24*30]
print int(float(len(averageTimesFiltered))/len(averageTimes)*100),"%"
plt.hist(averageTimesFiltered, bins = 30-0.5)

In [283]:
with open('data/categoryPairs.json', 'w') as outfile:
    json.dump(TMUsersCategoriesList, outfile)

In [195]:
from itertools import tee, izip
def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return izip(a, b)

In [77]:
len(allCheckins[['venueId', 'categoryId']].set_index('venueId').to_dict().keys())

1

In [94]:
np.unique(LDAclusteredUsersDF['clusterId'].tolist())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [101]:
LNG = allCheckins[['venueId', 'lat','lng']].set_index('venueId').to_dict()['lng']
LAT = allCheckins[['venueId', 'lat','lng']].set_index('venueId').to_dict()['lat']

In [ ]:
from itertools import izip
for (i, j), (k, v) in izip(LNG.iteritems(), LAT.iteritems()):
    print i, j, k, v

In [ ]:
allCheckins[['venueId', 'categoryId']].set_index('venueId').to_dict()['categoryId']

In [103]:
allCheckins.keys()

Index([u'categoryId', u'categoryName', u'checkinId', u'checkinsCount',
       u'createdAt', u'createdAtDate', u'lat', u'lng', u'name', u'tipCount',
       u'userId', u'usersCount', u'venueId'],
      dtype='object')

In [121]:
checkinsCountGrouped = []
for group in LDAclusteredUsersDF[LDAclusteredUsersDF.clusterId == 1].groupby(['hour','weekday']):
    checkinsCountGrouped.append({'key':group[0],
                               'value':len(group[1])})
checkinsCountGrouped

In [123]:
with open('testTime.json', 'w') as outfile:
        json.dump(checkinsCountGrouped, outfile)

In [52]:
topCheckins = pd.read_csv('data/July/LDAClusteredUsersMoreThan10.tsv', sep='\t')

In [53]:
len(set(topCheckins['name'].tolist()))

56477

In [39]:
set(topCheckins['clusterId'].tolist())

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}

In [40]:
topCheckins[['venueId', 'clusterId']].set_index('venueId').to_dict()['clusterId']

{'4b08d1d7f964a520a11223e3': 13,
 '4b4d5b48f964a52055d126e3': 5,
 '4b4f33fff964a52024fd26e3': 5,
 '4b5086ccf964a520212627e3': 18,
 '4b57605cf964a520873428e3': 11,
 '4b62c18df964a52069512ae3': 18,
 '4b63406af964a520a36d2ae3': 20,
 '4b645287f964a52095aa2ae3': 9,
 '4b71a2a7f964a52095522de3': 14,
 '4b7563d3f964a520d8092ee3': 13,
 '4b76c7bdf964a520a85e2ee3': 8,
 '4b76e9d3f964a520e7692ee3': 20,
 '4b827a36f964a52098d430e3': 18,
 '4b9379e6f964a520364534e3': 20,
 '4b937e60f964a520e34534e3': 6,
 '4b9bb751f964a520361d36e3': 11,
 '4b9cb499f964a520617736e3': 13,
 '4b9ce827f964a520b48136e3': 7,
 '4b9f2998f964a520161637e3': 13,
 '4ba1ba30f964a520cac937e3': 9,
 '4ba3a1c7f964a520064e38e3': 12,
 '4ba60394f964a520582e39e3': 9,
 '4ba7142cf964a520aa7f39e3': 20,
 '4ba754cdf964a520268c39e3': 20,
 '4ba765e4f964a520b09039e3': 9,
 '4ba8486cf964a520d7d439e3': 9,
 '4ba9c796f964a520c5373ae3': 20,
 '4bab6062f964a520a3a43ae3': 12,
 '4bab69a0f964a5206aa73ae3': 9,
 '4bab748cf964a520aeaa3ae3': 12,
 '4bade2cbf964a520067

In [41]:
venueByCluster = [0 for i in range(20)]
for clusterCheckins in topCheckins.groupby('clusterId'):
    names =  list(np.unique(clusterCheckins[1]['name'].tolist()))
    ids = list(np.unique(clusterCheckins[1]['venueId'].tolist()))
    venueByCluster[clusterCheckins[0] - 1] = {"names": names,
                                         "ids":ids}

In [43]:
with open('data/clustersTopVenues.json', 'w') as outfile:
        json.dump(venueByCluster, outfile)

In [63]:
categoryByCluster = [[] for i in range(20)]
for clusterCheckins in topCheckins.groupby('clusterId'):
    print clusterCheckins[0]
    for categoryId, group in clusterCheckins[1].groupby('categoryName').agg('size').order(ascending = False).iteritems():
        categoryByCluster[clusterCheckins[0]-1].append(categoryId)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [64]:
for index, category in enumerate(categoryByCluster):
    categoryByCluster[index] = categoryByCluster[index][:10]

In [65]:
with open('data/clustersTopCategories.json', 'w') as outfile:
        json.dump(categoryByCluster, outfile)